# Agente ReAct: Análisis Profundo

**Análisis Profesional del Framework de Razonamiento y Acción**

Este notebook proporciona un análisis completo de la implementación del framework ReAct (Reasoning + Acting) con:
- Representación visual de grafos de ejecución del agente
- Análisis paso a paso del trace de razonamiento
- Métricas de rendimiento y comparaciones
- Evaluación crítica de patrones de uso de herramientas

---

In [ ]:
# Importaciones principales
import sys
from pathlib import Path
import json
import time
from datetime import datetime
import os
from dotenv import load_dotenv
import asyncio

# Cargar variables de entorno
env_path = Path.cwd().parent / '.env'
load_dotenv(env_path)

# Agregar proyecto al path
sys.path.insert(0, str(Path.cwd().parent))

# Librerías de visualización
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import networkx as nx
from IPython.display import display, HTML, Markdown, Image

# Importaciones del proyecto
from src.agents.agent_factory import AgentFactory
from src.agents.agent_executor import AgentExecutor
from src.database.db_manager import DatabaseManager
from src.config import config

# Configuración de estilo
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

## 1. Resumen del Framework ReAct

### ¿Qué es ReAct?

ReAct combina **Razonamiento** (pensar qué hacer) y **Acción** (usar herramientas para hacerlo) en un framework unificado.

**Concepto Clave**: En lugar de solo generar texto, el LLM puede razonar sobre qué herramientas usar y cuándo, creando un proceso de toma de decisiones transparente.

### Bucle ReAct:

```
Consulta → [Pensamiento → Acción → Observación]* → Respuesta Final
```

Donde:
- **Pensamiento**: El LLM razona sobre qué hacer a continuación
- **Acción**: El LLM selecciona una herramienta y proporciona entrada
- **Observación**: La herramienta devuelve resultados al LLM
- **Repetir** hasta que el LLM tenga suficiente información para responder

---

## 2. Inicializar Agente y Herramientas

Probaremos con diferentes tipos de consultas para entender cómo se adapta ReAct:
1. **Consulta de Base de Datos**: Requiere buscar en la base de datos de Pink Floyd
2. **Consulta de Divisas**: Requiere llamar a la API externa de divisas
3. **Consulta Multi-Herramienta**: Requiere usar ambas herramientas

In [ ]:
# Inicializar el factory de agentes - usando tipo 'react' para evitar problemas con asyncio
factory = AgentFactory()

# Crear agente ReAct estándar (no CoT) para compatibilidad con Jupyter
model_name = "gpt-4o-mini"
agent = factory.create_agent(model_name, agent_type='react')
executor = AgentExecutor(agent, model_name)

# Mostrar herramientas disponibles
print("[HERRAMIENTAS] Herramientas Disponibles:")
print("="*80)
for i, tool in enumerate(factory.get_tools(), 1):
    print(f"{i}. {tool.name}")
    print(f"   Descripción: {tool.description[:100]}...")
    print()

print(f"[OK] Agente inicializado con modelo: {model_name}")
print(f"[OK] {len(factory.get_tools())} herramientas disponibles")

## 2.5 Visualización de Arquitectura con LangGraph

Además del agente ReAct estándar, crearemos un agente basado en LangGraph StateGraph.

**Diferencias Clave:**
- **NetworkX**: Muestra el trace de ejecución (qué pasó en una consulta específica)
- **LangGraph**: Muestra la arquitectura del agente (cómo está diseñado el sistema)

LangGraph permite visualizar la estructura del agente como un grafo de estado, mostrando:
- Nodos: agent (razonamiento), tools (ejecución)
- Edges: Flujo de control entre nodos
- Decisiones condicionales: Cuándo continuar o terminar

In [ ]:
# Crear agente LangGraph ReAct para visualización de arquitectura
langgraph_agent = factory.create_agent(model_name, agent_type='langgraph_react')

print(f"[OK] Agente LangGraph creado - Tipo: StateGraph - Herramientas: {len(factory.get_tools())}")
print("\n[VISUALIZACIÓN] Arquitectura del Agente ReAct con LangGraph")
print("="*80)

# Generar y mostrar el diagrama Mermaid del grafo
try:
    graph_image = langgraph_agent.get_graph().draw_mermaid_png()
    display(Image(graph_image))
    
    print("\n[OK] Diagrama generado exitosamente")
    print("\nEste diagrama muestra:")
    print("  • START: Punto de inicio")
    print("  • agent: Nodo de razonamiento (LLM)")
    print("  • tools: Nodo de ejecución de herramientas")
    print("  • END: Punto final")
    print("\nFlujo: START → agent → [tools → agent]* → END")
except Exception as e:
    print(f"[ERROR] No se pudo generar el diagrama: {e}")
    import traceback
    traceback.print_exc()

## 3. Funciones de Visualización

Definimos funciones para visualizar el trace de razonamiento del agente.

In [ ]:
def visualize_reasoning_trace(trace, title="Grafo de Ejecución ReAct"):
    """
    Crea una representación visual del trace de razonamiento ReAct.
    
    Usa NetworkX para construir un grafo dirigido mostrando:
    - Nodo Query (punto de inicio)
    - Nodos Thought (razonamiento)
    - Nodos Action (llamadas a herramientas)
    - Nodos Observation (resultados de herramientas)
    - Nodo Answer (salida final)
    """
    G = nx.DiGraph()
    
    # Mapeo de colores para diferentes tipos de nodos
    colors = {
        'query': '#FF1493',      # Rosa
        'thought': '#4169E1',    # Azul
        'action': '#32CD32',     # Verde
        'observation': '#FFA500',# Naranja
        'answer': '#9370DB'      # Púrpura
    }
    
    node_colors = []
    node_sizes = []
    labels = {}
    
    # Agregar nodo query
    G.add_node(0)
    node_colors.append(colors['query'])
    node_sizes.append(3000)
    labels[0] = "Consulta"
    
    prev_node = 0
    node_id = 1
    
    # Procesar pasos del trace
    for step in trace:
        step_type = step.get('type', '')
        
        if step_type in ['thought', 'action', 'observation']:
            G.add_node(node_id)
            G.add_edge(prev_node, node_id)
            
            # Establecer color y etiqueta según el tipo
            if step_type == 'action':
                node_colors.append(colors['action'])
                node_sizes.append(2500)
                tool = step.get('tool', 'desconocido')
                labels[node_id] = f"Acción\n{tool}"
            elif step_type == 'observation':
                node_colors.append(colors['observation'])
                node_sizes.append(2500)
                labels[node_id] = "Observación"
            else:
                node_colors.append(colors['thought'])
                node_sizes.append(2000)
                labels[node_id] = "Pensamiento"
            
            prev_node = node_id
            node_id += 1
    
    # Agregar nodo answer
    G.add_node(node_id)
    G.add_edge(prev_node, node_id)
    node_colors.append(colors['answer'])
    node_sizes.append(3000)
    labels[node_id] = "Respuesta"
    
    # Crear visualización
    plt.figure(figsize=(16, 10))
    
    # Usar layout jerárquico para mejor legibilidad
    pos = nx.spring_layout(G, k=2, iterations=50, seed=42)
    
    # Dibujar grafo
    nx.draw_networkx_nodes(G, pos, 
                          node_color=node_colors,
                          node_size=node_sizes,
                          alpha=0.9,
                          edgecolors='black',
                          linewidths=2)
    
    nx.draw_networkx_edges(G, pos,
                          edge_color='gray',
                          arrows=True,
                          arrowsize=25,
                          arrowstyle='->',
                          width=2.5,
                          connectionstyle='arc3,rad=0.1')
    
    nx.draw_networkx_labels(G, pos, labels,
                           font_size=10,
                           font_weight='bold')
    
    plt.title(title, fontsize=18, fontweight='bold', pad=20)
    plt.axis('off')
    plt.tight_layout()
    
    # Agregar leyenda
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=colors['query'], 
                   markersize=12, label='Consulta', markeredgecolor='black', markeredgewidth=1.5),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=colors['thought'], 
                   markersize=12, label='Pensamiento', markeredgecolor='black', markeredgewidth=1.5),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=colors['action'], 
                   markersize=12, label='Acción', markeredgecolor='black', markeredgewidth=1.5),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=colors['observation'], 
                   markersize=12, label='Observación', markeredgecolor='black', markeredgewidth=1.5),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=colors['answer'], 
                   markersize=12, label='Respuesta', markeredgecolor='black', markeredgewidth=1.5)
    ]
    plt.legend(handles=legend_elements, loc='upper right', fontsize=11, framealpha=0.9)
    
    plt.show()
    
    return G


def analyze_trace(trace, query):
    """
    Analiza el trace de razonamiento con evaluación crítica.
    """
    print(f"\n[CONSULTA] {query}")
    print("="*100)
    print()
    
    action_count = 0
    tools_used = set()
    
    for i, step in enumerate(trace, 1):
        step_type = step.get('type', '')
        
        if step_type == 'query':
            print(f"[CONSULTA] PASO {i}: CONSULTA DEL USUARIO")
            print(f"   Contenido: {step.get('content', '')[:100]}...")
            
        elif step_type == 'thought':
            print(f"\n[PENSAMIENTO] PASO {i}: RAZONAMIENTO")
            content = step.get('content', '')
            print(f"   {content[:250]}..." if len(content) > 250 else f"   {content}")
            
        elif step_type == 'action':
            action_count += 1
            tool = step.get('tool', 'desconocido')
            tools_used.add(tool)
            print(f"\n[ACCIÓN] PASO {i}: LLAMADA A HERRAMIENTA #{action_count}")
            print(f"   Herramienta: {tool}")
            print(f"   Entrada: {json.dumps(step.get('input', {}), indent=6, ensure_ascii=False)}")
            
        elif step_type == 'observation':
            print(f"\n[OBSERVACIÓN] PASO {i}: RESULTADO DE HERRAMIENTA")
            content = step.get('content', '')
            if len(content) > 300:
                print(f"   Resultado: {content[:300]}...")
                print(f"   (truncado - longitud total: {len(content)} caracteres)")
            else:
                print(f"   Resultado: {content}")
    
    print("\n" + "="*100)
    print("[RESUMEN]")
    print(f"  Total de Pasos: {len(trace)}")
    print(f"  Llamadas a Herramientas: {action_count}")
    print(f"  Herramientas Usadas: {', '.join(tools_used) if tools_used else 'Ninguna'}")
    print("="*100)


def display_result_summary(result, query_name):
    """
    Muestra un resumen visual del resultado.
    """
    print(f"\n\n{'='*100}")
    print(f"[CASO] {query_name}")
    print("="*100)
    print(f"\n[RESPUESTA]\n")
    print(result["answer"])
    print("\n" + "─"*100)
    
    metrics = result["metrics"]
    print("\n[MÉTRICAS]")
    print(f"  Tiempo de Ejecución: {metrics['execution_time_seconds']:.2f}s")
    print(f"  Tokens Usados: {metrics['estimated_tokens']['total']:,}")
    print(f"  Costo Estimado: ${metrics['estimated_cost_usd']:.6f}")
    print(f"  Pasos de Razonamiento: {metrics['num_steps']}")
    print("="*100)

## 4. Caso de Prueba 1: Consulta de Base de Datos

**Consulta**: "Encuentra canciones melancólicas de Pink Floyd de los años 70"

**Comportamiento Esperado**:
- El agente debe reconocer que requiere búsqueda en base de datos
- Debe usar la herramienta `pink_floyd_database`
- Debe parsear mood="melancholic" y década="1970s"
- Debe formatear resultados claramente

In [ ]:
# Ejecutar consulta de base de datos
query1 = "Encuentra canciones melancólicas de Pink Floyd de los años 70"

print(f"\n[EJECUTANDO] Consulta 1: {query1}")
print("[PROCESANDO]...\n")

# FIX: Use await directly in Jupyter
result1 = await executor.execute(query1)

# Mostrar resumen
display_result_summary(result1, "CASO 1: CONSULTA DE BASE DE DATOS")

### 4.1 Visualizar Trace de Razonamiento - Consulta de Base de Datos

In [ ]:
print("\n[VISUALIZACIÓN] Grafo de Ejecución:")
print("="*80)
graph1 = visualize_reasoning_trace(
    result1["reasoning_trace"],
    title="ReAct: Ejecución de Consulta de Base de Datos"
)

### 4.2 Análisis Detallado del Trace - Consulta de Base de Datos

In [ ]:
analyze_trace(result1["reasoning_trace"], query1)

### 4.3 Análisis Crítico - Consulta de Base de Datos

**Lo que funcionó bien:**
- El agente identificó correctamente la necesidad de usar la herramienta de base de datos
- Extrajo los parámetros relevantes (mood, década)
- Formateó los resultados de manera amigable para el usuario

**Posibles mejoras:**
- ¿Podría el agente haber sido más específico en la consulta?
- ¿Es suficiente una llamada a herramienta o debería verificar resultados?
- ¿Cómo se compara la calidad de la respuesta con la verdad de base?

---

## 5. Caso de Prueba 2: Consulta de Divisas

**Consulta**: "¿Cuál es el tipo de cambio actual de USD a EUR?"

**Comportamiento Esperado**:
- El agente debe reconocer que requiere datos de divisas
- Debe usar la herramienta `currency_price_checker`
- Debe parsear el par de divisas (USD/EUR)
- Debe proporcionar la tasa actual con contexto

In [ ]:
# Ejecutar consulta de divisas
query2 = "¿Cuál es el tipo de cambio actual de USD al Euro?"

print(f"\n[EJECUTANDO] Consulta 2: {query2}")
print("[PROCESANDO]...\n")

# FIX: Use await directly in Jupyter
result2 = await executor.execute(query2)

# Mostrar resumen
display_result_summary(result2, "CASO 2: CONSULTA DE DIVISAS")

### 5.1 Visualizar Trace de Razonamiento - Consulta de Divisas

In [ ]:
print("\n[VISUALIZACIÓN] Grafo de Ejecución:")
print("="*80)
graph2 = visualize_reasoning_trace(
    result2["reasoning_trace"],
    title="ReAct: Ejecución de Consulta de Divisas"
)

### 5.2 Análisis Detallado del Trace - Consulta de Divisas

In [ ]:
analyze_trace(result2["reasoning_trace"], query2)

---

## 6. Caso de Prueba 3: Consulta Multi-Herramienta

**Consulta**: "Quiero música energética de Pink Floyd para escuchar, y también dime el tipo de cambio de EUR a GBP"

**Comportamiento Esperado**:
- El agente debe reconocer DOS tareas distintas
- Debe usar AMBAS herramientas secuencialmente
- Debe combinar resultados coherentemente
- **Esto prueba la capacidad del agente para manejar consultas complejas de múltiples pasos**

In [ ]:
# Ejecutar consulta multi-herramienta
query3 = "Quiero música energética de Pink Floyd para escuchar, y también dime el tipo de cambio de EUR a GBP"

print(f"\n[EJECUTANDO] Consulta 3: {query3}")
print("[PROCESANDO]...\n")

# FIX: Use await directly in Jupyter
result3 = await executor.execute(query3)

# Mostrar resumen
display_result_summary(result3, "CASO 3: CONSULTA MULTI-HERRAMIENTA")

### 6.1 Visualizar Trace de Razonamiento - Consulta Multi-Herramienta

In [ ]:
print("\n[VISUALIZACIÓN] Grafo de Ejecución:")
print("="*80)
graph3 = visualize_reasoning_trace(
    result3["reasoning_trace"],
    title="ReAct: Ejecución de Consulta Multi-Herramienta"
)

### 6.2 Análisis Detallado del Trace - Consulta Multi-Herramienta

In [ ]:
analyze_trace(result3["reasoning_trace"], query3)

### 6.3 Análisis Crítico - Consulta Multi-Herramienta

**Observaciones Clave:**
1. ¿Identificó correctamente el agente ambas sub-tareas?
2. ¿Se llamaron las herramientas en orden lógico?
3. ¿Fue la respuesta final coherente y completa?
4. ¿Cuántos pasos adicionales de razonamiento se necesitaron vs consultas de herramienta única?

**Esto demuestra la capacidad del agente para:**
- Descomponer consultas complejas
- Coordinar múltiples herramientas
- Sintetizar información dispar

---

## 7. Análisis Comparativo

Comparemos el rendimiento de los tres casos de prueba

In [ ]:
# Recopilar métricas
comparison_data = {
    'Tipo de Consulta': ['Solo Base de Datos', 'Solo Divisas', 'Multi-Herramienta'],
    'Consulta': [
        query1[:45] + '...',
        query2[:45] + '...',
        query3[:45] + '...'
    ],
    'Tiempo (s)': [
        result1['metrics']['execution_time_seconds'],
        result2['metrics']['execution_time_seconds'],
        result3['metrics']['execution_time_seconds']
    ],
    'Total Tokens': [
        result1['metrics']['estimated_tokens']['total'],
        result2['metrics']['estimated_tokens']['total'],
        result3['metrics']['estimated_tokens']['total']
    ],
    'Costo (USD)': [
        result1['metrics']['estimated_cost_usd'],
        result2['metrics']['estimated_cost_usd'],
        result3['metrics']['estimated_cost_usd']
    ],
    'Pasos': [
        result1['metrics']['num_steps'],
        result2['metrics']['num_steps'],
        result3['metrics']['num_steps']
    ],
    'Herramientas': [
        len([s for s in result1['reasoning_trace'] if s.get('type') == 'action']),
        len([s for s in result2['reasoning_trace'] if s.get('type') == 'action']),
        len([s for s in result3['reasoning_trace'] if s.get('type') == 'action'])
    ]
}

df = pd.DataFrame(comparison_data)

# Mostrar tabla
print("\n[COMPARACIÓN] Comparación de Rendimiento:")
print("="*100)
display(df.style.background_gradient(cmap='Blues', subset=['Tiempo (s)', 'Total Tokens', 'Pasos']))

# Crear visualizaciones
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. Tiempo de ejecución
axes[0, 0].bar(df['Tipo de Consulta'], df['Tiempo (s)'], color=['#FF1493', '#4169E1', '#32CD32'], alpha=0.8)
axes[0, 0].set_title('Comparación de Tiempo de Ejecución', fontweight='bold', fontsize=12)
axes[0, 0].set_ylabel('Segundos')
axes[0, 0].set_ylim(0, max(df['Tiempo (s)']) * 1.2)
axes[0, 0].tick_params(axis='x', rotation=15)

# 2. Uso de tokens
axes[0, 1].bar(df['Tipo de Consulta'], df['Total Tokens'], color=['#FF1493', '#4169E1', '#32CD32'], alpha=0.8)
axes[0, 1].set_title('Comparación de Uso de Tokens', fontweight='bold', fontsize=12)
axes[0, 1].set_ylabel('Tokens')
axes[0, 1].tick_params(axis='x', rotation=15)

# 3. Costo
axes[1, 0].bar(df['Tipo de Consulta'], df['Costo (USD)'], color=['#FF1493', '#4169E1', '#32CD32'], alpha=0.8)
axes[1, 0].set_title('Comparación de Costo', fontweight='bold', fontsize=12)
axes[1, 0].set_ylabel('USD')
axes[1, 0].ticklabel_format(style='plain', axis='y')
axes[1, 0].tick_params(axis='x', rotation=15)

# 4. Complejidad (Pasos + Herramientas)
x = range(len(df))
width = 0.35
axes[1, 1].bar([i - width/2 for i in x], df['Pasos'], width, label='Pasos de Razonamiento', color='#4169E1', alpha=0.8)
axes[1, 1].bar([i + width/2 for i in x], df['Herramientas'], width, label='Herramientas Usadas', color='#32CD32', alpha=0.8)
axes[1, 1].set_title('Comparación de Complejidad', fontweight='bold', fontsize=12)
axes[1, 1].set_ylabel('Cantidad')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(df['Tipo de Consulta'])
axes[1, 1].tick_params(axis='x', rotation=15)
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# Estadísticas adicionales
print("\n[ESTADÍSTICAS] Resumen:")
print("="*100)
print(f"  Tiempo Promedio: {df['Tiempo (s)'].mean():.2f}s")
print(f"  Pasos Promedio: {df['Pasos'].mean():.1f}")
print(f"  Tokens Promedio: {df['Total Tokens'].mean():.0f}")
print(f"  Costo Promedio: ${df['Costo (USD)'].mean():.6f}")
print(f"  Herramientas Promedio: {df['Herramientas'].mean():.1f}")
print("="*100)

## 8. Insights Críticos

### 8.1 Fortalezas del Framework ReAct

1. **Transparencia**: Cada decisión es visible en el trace de razonamiento
2. **Selección de Herramientas**: El agente elige autónomamente las herramientas correctas
3. **Adaptabilidad**: Maneja consultas de herramienta única y multi-herramienta
4. **Refinamiento Iterativo**: Puede llamar herramientas múltiples veces si es necesario

### 8.2 Patrones Observados

**De nuestros experimentos:**

- **Consultas de base de datos**: Típicamente 1 llamada a herramienta, ejecución rápida
- **Consultas de divisas**: 1 llamada a herramienta, rendimiento similar
- **Consultas multi-herramienta**: 2+ llamadas a herramientas, proporcionalmente más tokens/tiempo

**Escalado de complejidad**:
- Aumento lineal en tokens con el número de herramientas
- El tiempo de ejecución crece sub-linealmente
- El costo escala predeciblemente con el uso de tokens

### 8.3 Consideraciones de Producción

**Debe implementarse:**
- Límites de iteración máxima (prevenir bucles infinitos)
- Validación de entrada (sanitizar entradas de herramientas)
- Manejo de errores (degradación elegante)
- Logging (depurar patrones de uso de herramientas)
- Limitación de tasa (prevenir abuso de API)

**Bueno tener:**
- Caché de llamadas a herramientas (evitar llamadas redundantes)
- Ejecución paralela de herramientas (acelerar consultas multi-herramienta)
- Puntuación de confianza (marcar respuestas inciertas)
- Bucle de retroalimentación del usuario (mejorar con el tiempo)

---

## 9. Verificación de Base de Datos

Verifiquemos los resultados de la consulta de base de datos para comprobar la precisión del agente

In [ ]:
# Verificar qué hay realmente en la base de datos
db_manager = DatabaseManager(config.database_path)

# Obtener canciones melancólicas de los años 70
melancholic_songs = db_manager.get_songs_by_mood("melancholic")
songs_1970s = [s for s in melancholic_songs if 1970 <= s.year <= 1979]

print("\n[VERIFICACIÓN] Verdad de Base: Canciones Melancólicas de Pink Floyd de los Años 70")
print("="*100)
print(f"Se encontraron {len(songs_1970s)} canciones:\n")

for song in songs_1970s:
    print(f"  {song.title} ({song.year}) - {song.album}")

print("\n" + "="*100)
print("\n[OK] Verificación de Precisión de la Respuesta del Agente:")
print("Compara la respuesta del agente arriba con esta verdad de base.")
print("¿El agente:")
print("  [ ] ¿Encontró todas las canciones?")
print("  [ ] ¿Incluyó solo canciones de los años 70?")
print("  [ ] ¿Incluyó solo canciones melancólicas?")
print("  [ ] ¿Formateó los resultados claramente?")
print("="*100)

---

## 11. Comparación de Visualizaciones: NetworkX vs LangGraph

Esta sección compara los dos enfoques de visualización que hemos utilizado en este análisis.

In [ ]:
print("="*100)
print("COMPARACIÓN: Dos Enfoques de Visualización para Agentes ReAct")
print("="*100)

print("\n### 1. NetworkX - Visualización de Trace de Ejecución\n")
print("Propósito:")
print("  - Muestra QUÉ PASÓ durante una ejecución específica del agente")
print("  - Captura el flujo real de razonamiento paso a paso")
print("  - Visualiza el trace post-ejecución")

print("\nCaracterísticas:")
print("  - Nodos con código de color por tipo (consulta, pensamiento, acción, observación)")
print("  - Muestra llamadas a herramientas específicas y sus resultados")
print("  - Cada visualización es única para cada consulta")
print("  - Útil para debugging y análisis de casos específicos")

print("\nCuándo usar:")
print("  - Para analizar por qué un agente tomó decisiones específicas")
print("  - Para depurar comportamiento inesperado del agente")
print("  - Para auditar el proceso de razonamiento")
print("  - Para comprender el flujo de una consulta compleja")

print("\n" + "-"*100)

print("\n### 2. LangGraph - Visualización de Arquitectura\n")
print("Propósito:")
print("  - Muestra CÓMO ESTÁ DISEÑADO el agente a nivel arquitectónico")
print("  - Revela la estructura StateGraph subyacente")
print("  - Visualiza el flujo de control del sistema")

print("\nCaracterísticas:")
print("  - Diagrama Mermaid de la arquitectura del grafo de estado")
print("  - Muestra nodos, edges y decisiones condicionales")
print("  - Igual para todas las ejecuciones (muestra el diseño, no la ejecución)")
print("  - Generado desde la definición del grafo")

print("\nComponentes del grafo:")
print("  - START: Punto de entrada del sistema")
print("  - agent: Nodo donde el LLM razona y decide acciones")
print("  - tools: Nodo donde se ejecutan las herramientas")
print("  - Conditional edges: Lógica de enrutamiento (continuar/terminar)")
print("  - END: Punto de salida del sistema")

print("\nCuándo usar:")
print("  - Para entender la estructura del agente antes de ejecutarlo")
print("  - Para documentar arquitectura de sistemas agénticos")
print("  - Para comunicar diseño a otros desarrolladores")
print("  - Para identificar posibles loops o deadlocks en el diseño")

print("\n" + "="*100)
print("\n### Resumen de Diferencias Clave\n")

comparison_table = {
    'Aspecto': [
        'Qué muestra',
        'Cuándo se genera',
        'Varía por consulta',
        'Nivel de abstracción',
        'Uso principal',
        'Formato'
    ],
    'NetworkX (Trace)': [
        'Flujo de ejecución real',
        'Después de ejecutar consulta',
        'Sí - cada consulta diferente',
        'Concreto - pasos específicos',
        'Debugging y análisis',
        'Grafo dirigido con NetworkX'
    ],
    'LangGraph (Arquitectura)': [
        'Diseño del sistema',
        'Desde definición del grafo',
        'No - siempre igual',
        'Abstracto - estructura del agente',
        'Documentación y diseño',
        'Diagrama Mermaid'
    ]
}

df_comparison = pd.DataFrame(comparison_table)
display(df_comparison)

print("\n" + "="*100)
print("\n[CONCLUSIÓN]")
print("Ambas visualizaciones son complementarias:")
print("  - LangGraph: Entender CÓMO funciona el agente (arquitectura)")
print("  - NetworkX: Entender QUÉ hizo el agente (comportamiento)")
print("="*100)

---

## 10. Conclusiones

### Aprendizajes Clave

1. **ReAct proporciona transparencia**: Cada decisión es rastreable
2. **La selección de herramientas es confiable**: El agente elige consistentemente las herramientas correctas
3. **El rendimiento es predecible**: El costo y el tiempo escalan linealmente con la complejidad
4. **Las consultas multi-herramienta funcionan**: El agente puede coordinar múltiples herramientas efectivamente

### Cuándo Usar ReAct

**Bueno para:**
- Tareas que requieren datos/herramientas externos
- Situaciones que necesitan transparencia/explicabilidad
- Flujos de trabajo de razonamiento multi-paso
- Escenarios de selección dinámica de herramientas

**No ideal para:**
- Generación pura de texto (no se necesitan herramientas)
- Aplicaciones críticas de latencia (múltiples llamadas a LLM)
- Tareas simples de clasificación (excesivo)
- Cuando la confiabilidad de herramientas es incierta

### Próximos Pasos

1. **Agregar más herramientas**: Expandir capacidades del agente
2. **Implementar caché**: Reducir llamadas redundantes a herramientas
3. **Agregar validación**: Verificar resultados de herramientas antes de usar
4. **Monitorear en producción**: Rastrear modos de fallo
5. **Pruebas A/B de modelos**: Encontrar balance óptimo costo/rendimiento

---

### Referencias:
- Paper ReAct: https://arxiv.org/abs/2210.03629
- Agentes LangChain: https://python.langchain.com/docs/modules/agents/

---

**Análisis Completado**

Este notebook demostró el poder y la transparencia del framework ReAct para construir agentes inteligentes que pueden razonar y actuar de manera coordinada.

In [ ]:
print("="*100)
print("[LANGGRAPH DEV] Servidor de Desarrollo LangGraph - ReAct Agent")
print("="*100)

print("\nEl servidor de desarrollo LangGraph permite:")
print("  • Visualización interactiva del grafo en tiempo real")
print("  • Debugging paso a paso de la ejecución del agente")
print("  • Inspección de estado en cada nodo")
print("  • Timeline de ejecución")

print("\n[INSTRUCCIONES] Para iniciar el servidor:")
print("="*100)

print("\n1. En una terminal, navega al directorio del proyecto:")
print("   cd /Users/carlosdaniel/Desktop/henry_clase_demo")

print("\n2. Inicia el servidor LangGraph para ReAct Agent (puerto 8123):")
print("   langgraph dev --port 8123")

print("\n3. Abre tu navegador en:")
print("   http://localhost:8123")

print("\n[NOTA] Para el notebook de Cinema, usa el puerto 8124")
print("="*100)

## 11. Integración con LangGraph Dev Server

El servidor de desarrollo LangGraph permite visualización interactiva y debugging paso a paso.